In [34]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [35]:
## Create  SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkConf
spark = SparkSession.builder\
        .appName("SparkDataFrames_and_SQL_on_Sales_Dataset")\
        .master('local[*]')\
        .enableHiveSupport()\
        .config('spark.sql.warehouse.dir','hdfs://bigdata:8020/user/rupesh/spark-warehouse')\
        .getOrCreate()

In [36]:
spark

#### __**Data is available in HDFS as csv file at**__ 
<font color='blue'>**‘/user/jayantm/Batch48/CUTe/PartB/sales_tf_data.csv’**</font>


####  **Submission file:**
Submission file should be a zip file with name: 20190120_CSE7322c_Batch48_cuTE_PartB_EnrollID.zip
Zip file should consist of below files:
1.	Ipynb file or html file containing all the steps described above. (Steps 1 to Steps 13)


**Data description:** <br/><br/>
Data file contains the data for the orders for which invoices have been generated. Each Order has been processed or cancelled based on Invoice Number.<br/> 
    <font color='brown'>_InvoiceNumber_</font> – Invoice Number of the order. If Order starts with “C”, it means cancelled.<br/>
    <font color='brown'>_StockCode_</font> – Stock Code of the Product.   <br/>
    <font color='brown'>_Description_</font> – Description of the product (if available) <br/>
    <font color='brown'>_Quantity_</font> – Quantity of the Product Ordered. <br/>
    <font color='brown'>_Invoice Date_</font> – The Date on which the invoice was raised for the product.<br/>
    <font color='brown'>_Unit Price_</font> – The unit price of each product. <br/>
    <font color='brown'>_CustomerID_</font> – Customer id for which the order was raised. <br/>
    <font color='brown'>_Country_</font> – The country from where the Order was raised. <br/>


#### 1.	Copy the  data into the linux machine

In [37]:
!pwd

/nfsroot/data/home/2339B48/cuteB


In [38]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

In [50]:
!hdfs dfs -get /user/jayantm/Batch48/CUTe/PartB/sales_tf_data.csv /home/2339B48/cuteB/invoice

get: `/home/2339B48/cuteB/invoice/sales_tf_data.csv': File exists


#### 2. Using Spark Session read the csv data as a dataframe.  

In [86]:
data = spark.read.format("csv").option("header", "true").load("file:///home/2339B48/cuteB/invoice/sales_tf_data.csv*/")
data.show(3)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 3 rows



#### 3. Drop null values in the dataframe , if any 

In [87]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()
data = data.na.drop( how = 'any' )

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|        0|        0|        298|       0|          0|        0|     34101|      0|
+---------+---------+-----------+--------+-----------+---------+----------+-------+



#### 4. Create a new column in the dataframe called ‘Date’ .Convert Date to String format using the below 
#### Hint : to_date,from_unixtime,unix_timestamp


In [88]:
from pyspark.sql import functions as F
data = data.withColumn('date', F.to_date(data.InvoiceDate, "dd/MM/yyyy hh:mm"))
data.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-01-12|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-01-12|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|U

#### 5. Derive Day of the week from Date column created in Step 4.

In [89]:
from pyspark.sql.functions import date_format
data = data.withColumn('Day_Of_Week', date_format('date', 'u').alias('1'))
data.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|Day_Of_Week|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-01-12|          2|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-01-12|          2|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|     

#### 6. Derive Month column from Date column created in Step 4.

In [90]:
data = data.withColumn('Month', date_format('date', 'MM').alias('1'))
data.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|Day_Of_Week|Month|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-01-12|          2|   01|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|   01|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-01-12|          2|   01|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|   01|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|

#### 7. Group the data based on CustomerID and Date column

In [91]:
customer_date = data.groupby('CustomerID','Date')

customer_date_final = cust_date.count()
customer_date_final.show(4)

+----------+----------+-----+
|CustomerID|      Date|count|
+----------+----------+-----+
|     12583|2010-01-12|   20|
|     12755|2010-09-12|    1|
|     18065|2010-10-12|   21|
|     16875|2011-04-01|   22|
+----------+----------+-----+
only showing top 4 rows



#### 8. Group the data based on CustomerID again and filter the data for only those records whose count is greater than 39(thirty nine). Rename the CustomerID column to ‘CustID’ 

In [92]:
customer_count = data.groupby('CustomerID').count()
customer_count = customer_count.where('count > 39')
customer_count = customer_count.withColumnRenamed('CustomerID','CustID')
customer_count.show(4)

+------+-----+
|CustID|count|
+------+-----+
| 15574|  159|
| 15555|   96|
| 15271|   40|
| 17686|   68|
+------+-----+
only showing top 4 rows



#### 9. Join the dataframes created in Step 8 and Step 7 using the CustID and CustomerID.

In [93]:
joinExpression = customer_count["CustID"] == customer_date_final["CustomerID"]
customer_date_final.join(customer_count, joinExpression).show(4)

+----------+----------+-----+------+-----+
|CustomerID|      Date|count|CustID|count|
+----------+----------+-----+------+-----+
|     12583|2010-01-12|   20| 12583|   56|
|     16875|2011-04-01|   22| 16875|   52|
|     15574|      null|  159| 15574|  159|
|     15555|      null|   96| 15555|   96|
+----------+----------+-----+------+-----+
only showing top 4 rows



#### 10. Derive new column ‘Val’ by computing the product of ‘Quantity’ column and ‘UnitPrice’ column. Round the ‘Val’ column to 2 values.

In [94]:
from pyspark.sql.functions import round
data = data.withColumn('Val',round((data.Quantity * data.UnitPrice),2))
data.show(4)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+-----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|Day_Of_Week|Month|  Val|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-01-12|          2|   01| 15.3|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|   01|20.34|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|2010-01-12|          2|   01| 22.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|   01|20.34|
+---------+---------+-------------

#### 11.Derive a new column ‘Label’ as binary value based on the Invoice Number.

In [95]:
data.createOrReplaceTempView("data")
data = spark.sql('SELECT *,CASE WHEN InvoiceNo LIKE "C%" THEN "Cancel" ELSE "Active" END AS Label FROM data')
data.show(2)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+-----+------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|      date|Day_Of_Week|Month|  Val| Label|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+-----+------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|2010-01-12|          2|   01| 15.3|Active|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|2010-01-12|          2|   01|20.34|Active|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+-----------+-----+-----+------+
only showing top 2 rows



#### 12. Check for which CustomerID are the most number of cancellations.

In [96]:
cancellations = data.filter('InvoiceNo like "C%"')

cancellations_df = cancel.groupby('InvoiceNo','CustomerID').count()

cancellations_df.orderBy(cancellations_df['count'].desc()).show(1)

+---------+----------+-----+
|InvoiceNo|CustomerID|count|
+---------+----------+-----+
|  C538341|     15514|   39|
+---------+----------+-----+
only showing top 1 row



#### 13. Select only Month,StockCode,DayofTheWeek and label in a dataframe.

In [99]:
finalmsdf_df = data.select('Month','StockCode','Day_Of_Week','Description')
finalmsdf_df.show(2)

+-----+---------+-----------+--------------------+
|Month|StockCode|Day_Of_Week|         Description|
+-----+---------+-----------+--------------------+
|   01|   85123A|          2|WHITE HANGING HEA...|
|   01|    71053|          2| WHITE METAL LANTERN|
+-----+---------+-----------+--------------------+
only showing top 2 rows

